In [64]:
#First, load in the necessary libraries - we're going to use Pandas to explore and preprocess the dataset
import pandas as pd
import numpy as np
import itertools
pd.set_option('display.max_columns', None)

In [65]:
#Next, load in the dataset from the url below. After doing so, print out each column and it's related data type (dtype) as well as a small data sample
#The dataset is a breakdown of charges to an individual's insurance, and includes characteristics of the individual that are age, sex, bmi, # of children, smoker, and region.
df = pd.read_csv('https://raw.githubusercontent.com/stedy/Machine-Learning-with-R-datasets/master/insurance.csv')
display(df.head())
print('-----------------')
for c in df:
    print(c, df[c].dtype)


,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


-----------------
age int64
sex object
bmi float64
children int64
smoker object
region object
charges float64


We can see each column above, and how each columns looks for this dataset. It looks like one thing we want to change is the column "smoker", which is a boolean yes/no. Let's cast this to an actual boolean datatype so it's easier to work with and acts as a boolean. Also, let's round the charges so it's easier to understand the values, as the number of digits after the decimal isn't as important for visualization purposes.

In [66]:
df['smoker'] = df['smoker'].replace({'yes': True, 'no': False})
df['charges'] = df['charges'].round(1)
display(df.head())

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,True,southwest,16884.9
1,18,male,33.770,1,False,southeast,1725.6
2,28,male,33.000,3,False,southeast,4449.5
3,33,male,22.705,0,False,northwest,21984.5
4,32,male,28.880,0,False,northwest,3866.9


Afer loading the data and making some small adjustments, let's explore the data a bit to see how each column relates to the "charges" column.

In [67]:
#This function takes in a string 'col' to group the dataset on, then gets the average charges for each group and displays the output
def groupby_object(col: str) -> None:
    col_groupby = df.groupby(col)['charges'].mean()
    display(col_groupby)
    print('-----------------')


#Explore each non continuous column
groupby_object(col='sex')
groupby_object(col='children')
groupby_object(col='smoker')
groupby_object(col='region')

sex
female    12569.578701
male      13956.751479
Name: charges, dtype: float64

-----------------


children
0    12365.974564
1    12731.173457
2    15073.565417
3    15355.315287
4    13850.664000
5     8786.038889
Name: charges, dtype: float64

-----------------


smoker
False     8434.268327
True     32050.232117
Name: charges, dtype: float64

-----------------


region
northeast    13406.385494
northwest    12417.576923
southeast    14735.410989
southwest    12346.935692
Name: charges, dtype: float64

-----------------


We have some interesting results here - at a glance, we can see that men spend are billed more on healthcare than women are. For number of children, we can see that individuals with more children are billed more on health insurance, except for 4 and 5 kids, where they are billed less which is interesting. Being a smoker is the strongest different for charges, where non smokers are billed nearly 1/4 as much as smokers are. Lastly we see region, where the southeast and northeast are billed slightly more than the west.

In [68]:
#Now, let's take a look at the continuous columns
#This function takes in a string 'col' to group the dataset on, as well as the number of bins to use, then gets the average charges for each group and displays the output
def groupby_continuous(col: str, num_bins: int) -> None:
    df[f'{col}_bins'] = pd.cut(df[col], bins=num_bins)
    col_groupby = df.groupby(f'{col}_bins')['charges'].mean()
    display(col_groupby)
    print('-----------------')

groupby_continuous(col='age', num_bins=10)
groupby_continuous(col='bmi', num_bins=10)

age_bins
(17.954, 22.6]     8375.016216
(22.6, 27.2]      10244.946429
(27.2, 31.8]      10589.781651
(31.8, 36.4]      11332.895313
(36.4, 41.0]      11829.646512
(41.0, 45.6]      15737.669091
(45.6, 50.2]      15013.604861
(50.2, 54.8]      17175.809649
(54.8, 59.4]      16077.639844
(59.4, 64.0]      21248.025439
Name: charges, dtype: float64

-----------------


bmi_bins
(15.923, 19.677]     9110.036667
(19.677, 23.394]     9596.429134
(23.394, 27.111]    11216.515385
(27.111, 30.828]    11802.999060
(30.828, 34.545]    14316.472917
(34.545, 38.262]    17968.272821
(38.262, 41.979]    14542.880208
(41.979, 45.696]    17405.068750
(45.696, 49.413]    17558.364286
(49.413, 53.13]     16034.333333
Name: charges, dtype: float64

-----------------


We can see strong correlations with both `age` and `bmi` - the higher for either one, the greater the charges that were billed, which makes sense.

In [69]:
#First, let's create categorical bins for the continuous variables
display(df.corr())

,age,bmi,children,smoker,charges
age,1.000000,0.109272,0.042469,-0.025019,0.299008
bmi,0.109272,1.000000,0.012759,0.003750,0.198341
children,0.042469,0.012759,1.000000,0.007673,0.067998
smoker,-0.025019,0.003750,0.007673,1.000000,0.787251
charges,0.299008,0.198341,0.067998,0.787251,1.000000


While we do see some strong correlations listed, it would make more sense to see `age` and `bmi` more closely linked to charges. However, this is likely due to the way correlation is calculated and having `smoker` as such a dominant correlation. What might be good to do next is do higher dimensional correlations

In [72]:
#This function groups data by 2 columns and sorts based on average charge for that group
def groupby_2dim(col1: str, col2: str) -> None:
    col_groupby = df.groupby([col1, col2])['charges'].mean()
    col_groupby = col_groupby.to_frame().reset_index(drop=False).sort_values(by='charges', ascending=False).dropna(how='any')
    display(col_groupby.head(10), col_groupby.tail(10))
    print('-----------------')

cols = ['age_bins', 'bmi_bins', 'children', 'smoker', 'region']
for comb in itertools.combinations(cols, 2):
    print(comb)
    groupby_2dim(col1=comb[0], col2=comb[1])

('age_bins', 'bmi_bins')


,age_bins,bmi_bins,charges
57,"(41.0, 45.6]","(41.979, 45.696]",46201.000000
58,"(41.0, 45.6]","(45.696, 49.413]",45863.200000
87,"(54.8, 59.4]","(41.979, 45.696]",30125.800000
77,"(50.2, 54.8]","(41.979, 45.696]",29437.300000
7,"(17.954, 22.6]","(41.979, 45.696]",26554.933333
48,"(36.4, 41.0]","(45.696, 49.413]",26274.550000
17,"(22.6, 27.2]","(41.979, 45.696]",25855.380000
94,"(59.4, 64.0]","(30.828, 34.545]",25551.041379
95,"(59.4, 64.0]","(34.545, 38.262]",24537.050000
78,"(50.2, 54.8]","(45.696, 49.413]",24412.550000


,age_bins,bmi_bins,charges
47,"(36.4, 41.0]","(41.979, 45.696]",6056.850000
6,"(17.954, 22.6]","(38.262, 41.979]",5518.523529
2,"(17.954, 22.6]","(23.394, 27.111]",4836.916129
1,"(17.954, 22.6]","(19.677, 23.394]",4830.760606
38,"(31.8, 36.4]","(45.696, 49.413]",4686.400000
27,"(27.2, 31.8]","(41.979, 45.696]",4509.900000
20,"(27.2, 31.8]","(15.923, 19.677]",3732.600000
0,"(17.954, 22.6]","(15.923, 19.677]",3622.157143
18,"(22.6, 27.2]","(45.696, 49.413]",3310.250000
19,"(22.6, 27.2]","(49.413, 53.13]",2438.100000


-----------------
('age_bins', 'children')


,age_bins,children,charges
58,"(59.4, 64.0]",4,36580.300000
55,"(59.4, 64.0]",1,26540.284615
56,"(59.4, 64.0]",2,25981.822222
57,"(59.4, 64.0]",3,23608.345455
45,"(50.2, 54.8]",3,21847.269565
44,"(50.2, 54.8]",2,21709.385714
49,"(54.8, 59.4]",1,19448.469565
54,"(59.4, 64.0]",0,19339.283750
50,"(54.8, 59.4]",2,18626.213333
28,"(36.4, 41.0]",4,17921.500000


...

,children,region,charges
0,0,northeast,11626.463265
17,4,northwest,11347.033333
1,0,northwest,11324.371212
7,1,southwest,10406.487179
15,3,southwest,10402.437838
5,1,northwest,10230.259459
22,5,southeast,10115.433333
21,5,northwest,8965.800000
23,5,southwest,8444.162500
20,5,northeast,6979.000000


-----------------
('smoker', 'region')


,smoker,region,charges
6,True,southeast,34844.995604
7,True,southwest,32269.063793
5,True,northwest,30192.006897
4,True,northeast,29673.535821
0,False,northeast,9165.533074
1,False,northwest,8556.464794
2,False,southeast,8032.216117
3,False,southwest,8019.282397


,smoker,region,charges
6,True,southeast,34844.995604
7,True,southwest,32269.063793
5,True,northwest,30192.006897
4,True,northeast,29673.535821
0,False,northeast,9165.533074
1,False,northwest,8556.464794
2,False,southeast,8032.216117
3,False,southwest,8019.282397


-----------------


Looking through each chart can show some pretty interesting trends - as we see with 1 dimensional grouping, `bmi`, `age`, and `smoker` are strongly correlated to higher charges for medical costs. Similarly, `children` does not seem to have a direct linear relationship with charges - groups with 4 or 5 children are among the lowest billed on average.